In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split

In [6]:
file_name = 'predictive_maintenance.csv'
df = pd.read_csv(file_name)
df.drop(['UDI','Product ID'], axis=1, inplace=True)
df = pd.get_dummies(df, dtype=int)
df.drop('Failure Type_No Failure', axis=1, inplace=True)
df.rename(columns={"Target":"Failure"}, inplace=True)
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure,Type_H,Type_L,Type_M,Failure Type_Heat Dissipation Failure,Failure Type_Overstrain Failure,Failure Type_Power Failure,Failure Type_Random Failures,Failure Type_Tool Wear Failure
0,298.1,308.6,1551,42.8,0,0,0,0,1,0,0,0,0,0
1,298.2,308.7,1408,46.3,3,0,0,1,0,0,0,0,0,0
2,298.1,308.5,1498,49.4,5,0,0,1,0,0,0,0,0,0
3,298.2,308.6,1433,39.5,7,0,0,1,0,0,0,0,0,0
4,298.2,308.7,1408,40.0,9,0,0,1,0,0,0,0,0,0


In [11]:
x_cols = ['Air temperature [K]', 'Process temperature [K]','Rotational speed [rpm]',
          'Torque [Nm]', 'Tool wear [min]','Type_H','Type_L', 'Type_M']

y_cols = ['Failure']

x = df[x_cols]
y = df[y_cols]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True, random_state=42)

train_indices = x_train.index
test_indices = x_test.index

model = RandomForestClassifier(random_state=42)
model.fit(x_train, y_train.values.ravel())
y_pred_prob = model.predict_proba(x_test) # It is (n,2) array where first number represents the probability that machine will not fail and second number represents probability that machine will fail
print(y_pred_prob)
# Now then again based on model, predict the outputs for rest failure types in terms of 0 and 1. Then whichever has output will be possible failure. Give suggestions on webapp based on that.
# This will involve training model based on same dataset twice but in second case, model will have different kind of output. Resembles to model stacking
y_cols_failure_type = ['Failure Type_Heat Dissipation Failure', 'Failure Type_Overstrain Failure', 'Failure Type_Power Failure', 'Failure Type_Random Failures', 'Failure Type_Tool Wear Failure']
y_failure_type = df[y_cols_failure_type]

x_train2 = x.loc[train_indices]
x_test2 = x.loc[test_indices]
y_train2 = y_failure_type.loc[train_indices]
y_test2 = y_failure_type.loc[test_indices]

model2 = RandomForestClassifier(random_state=42)
model2.fit(x_train2, y_train2)

y_pred_fail_type = model2.predict(x_test2)
print(y_pred_fail_type)

[[1.   0.  ]
 [0.99 0.01]
 [1.   0.  ]
 ...
 [1.   0.  ]
 [1.   0.  ]
 [1.   0.  ]]
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 ...
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
